In [8]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
#eval interval = 2500
learning_rate = 3e-4

cuda


In [9]:
with open('frankenstein_prometheus.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '#', '$', '%', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'æ', 'è', 'é', 'ê', 'ô', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [10]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
#print(data[:100])

In [11]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:1+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

tensor([230127, 217399, 112300, 171708])
inputs:
tensor([[59, 54, 71,  1, 54, 72,  1, 34],
        [61, 73, 58, 57,  1, 73, 61, 58],
        [ 1, 76, 54, 71, 66, 58, 57,  1],
        [68, 66,  1, 73, 61, 78,  1, 63]], device='cuda:0')
targets:
tensor([], device='cuda:0', size=(4, 0), dtype=torch.int64)


In [12]:


x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([92]) target is tensor(45)
when input is tensor([92, 45]) target is tensor(61)
when input is tensor([92, 45, 61]) target is tensor(58)
when input is tensor([92, 45, 61, 58]) target is tensor(1)
when input is tensor([92, 45, 61, 58,  1]) target is tensor(41)
when input is tensor([92, 45, 61, 58,  1, 41]) target is tensor(71)
when input is tensor([92, 45, 61, 58,  1, 41, 71]) target is tensor(68)
when input is tensor([92, 45, 61, 58,  1, 41, 71, 68]) target is tensor(63)


In [13]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indicies in current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


-êq]K1ôwxLy%Dé/0WE”Xa86uV‘a-od10txTfêEz“u_,rm0sDxehANk18D6Xwêqhtu,r—cqg2kB;Aêtfm(uLê“2?u™•aY”fXO]™cæ2 hYtD%Q8OYêXQoBQP]xdVYtp#0$)qKVL$:èAUa[K:eI[ioRglV.d$B]bv-è]xBEht
 IT,fæIVê$﻿U2HEPgGiu)z#;p[KO﻿O
0zyfæ”/æ9:#c’?V”2?Y8•uV]æ”9w™xw1™I
,oPn bQe‘t!0sæô_au?wLAêTEj88d*P_T—_aEqCMht9

JJ;$O.o$cypYexv’-9zkX!3sBu﻿™cCQtæupO.6Vônjc—æ_uhzFdgHOhA!é/e﻿XzmG%[K7“—LS,fêIim#9:è1:è[K:VrXW7p3;prBæn xêvml—VL1a[KqVNb_Vtêéqqt;N‘•mp-uV:akiehGv.ô9vxF3NUHk2yP$ô_T?’S#105uêP[K:yIx™IVVooMiQy2?﻿4ô$QèW3j9OQWUa8_”ygI/e(’,5u ê4*


In [14]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

NameError: name 'eval_iters' is not defined

In [15]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


oMoQai
DX‘K:9::V8:V”]SprlK•ss]™_5KblVL?2A_VVsED——U!k:ôsd#oMD:y”/%0b!xw[”HWIu•P2æNbv.c9læ8:)mæ#z—e2?vh)8NJD—uxjc.?’9ôCj*2?yYt9oykcQ[Y4-A0$z“5a:Pk,æ3é7(™jW*/yæMjaPSB_VjO:?‘mFf3%
aT_#A;A9
-Srbyjg•!B-Lkq!—D10*vxe?zDéht44R”:ô3hNXlv(Y;B,,é7R3FêOY”L4b“BôI(jkkXW%h2’_u—(!’èF3.ôJ6—H.ælV‘KE[DBE﻿’m6-km6c7[ée—_.rNb0u,IiNbto1“læT,wV9W?z8_d‘éJS﻿M6sGiU:yJk•h’B,IAoé9
0q’b_]]3éC!:êCQMvxê (™9W4xryBh1j’$RArc— htT﻿ y7$cNRPIw7U,?vmNWD4æô)S5!è”$•o/
_S5e)mWN.sff%If?;#D™gAl$—.6#s;A8cY8lBcFFCHo7g™pWIKfRæv/!JY8:f,2æô“1!1o
